# HRV Pattern Recognition with CNN for Apple Watch Data
This notebook implements a Convolutional Neural Network for detecting irregular heart patterns from Heart Rate Variability (HRV) time series data collected from Apple Watch continuous heart rate monitoring.

## Key Features:
- Uses HRV time series instead of ECG waveforms
- Adapted CNN architecture for HRV pattern recognition
- Detects irregular heart patterns from HRV variability
- Optimized for Apple Watch continuous heart rate data
- Multi-class classification for different HRV patterns

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")

## Step 1: Generate Synthetic Apple Watch HRV Data

We'll create realistic HRV time series data that mimics Apple Watch continuous heart rate monitoring patterns. The data will include:
- Normal HRV patterns (healthy variability)
- Atrial Fibrillation patterns (irregular intervals)
- Bradycardia patterns (slow heart rate with high variability)
- Tachycardia patterns (fast heart rate with low variability)

In [ ]:
class HRVDataGenerator:
    def __init__(self, sequence_length=200, sampling_rate=4):
        """
        Initialize HRV data generator for Apple Watch-like data
        
        Args:
            sequence_length: Number of RR intervals in each sequence
            sampling_rate: Approximate sampling rate in Hz (Apple Watch ~4Hz)
        """
        self.sequence_length = sequence_length
        self.sampling_rate = sampling_rate
        
    def generate_normal_hrv(self, base_hr=70):
        """
        Generate normal HRV pattern with healthy variability
        - Regular rhythm with natural variation
        - RMSSD typically 20-50ms
        - Heart rate around 60-100 bpm
        """
        # Convert HR to RR intervals (in milliseconds)
        base_rr = 60000 / base_hr  # Base RR interval in ms
        
        # Add respiratory sinus arrhythmia (natural variation)
        respiratory_freq = 0.25  # 15 breaths per minute
        time_points = np.linspace(0, self.sequence_length/self.sampling_rate, self.sequence_length)
        respiratory_variation = 30 * np.sin(2 * np.pi * respiratory_freq * time_points)
        
        # Add random variation
        random_variation = np.random.normal(0, 15, self.sequence_length)
        
        # Combine variations
        rr_intervals = base_rr + respiratory_variation + random_variation
        
        # Ensure physiologically reasonable values (300-2000ms)
        rr_intervals = np.clip(rr_intervals, 300, 2000)
        
        return rr_intervals
    
    def generate_afib_hrv(self, base_hr=85):
        """
        Generate atrial fibrillation HRV pattern
        - Highly irregular rhythm
        - Large variations in RR intervals
        - No clear pattern
        """
        base_rr = 60000 / base_hr
        
        # Create highly irregular pattern
        irregular_variation = np.random.normal(0, 80, self.sequence_length)
        
        # Add some clustering of short and long intervals
        cluster_variation = np.zeros(self.sequence_length)
        for i in range(0, self.sequence_length, 20):
            cluster_size = min(np.random.randint(3, 8), self.sequence_length - i)
            cluster_variation[i:i+cluster_size] = np.random.normal(0, 60, cluster_size)
        
        rr_intervals = base_rr + irregular_variation + cluster_variation
        rr_intervals = np.clip(rr_intervals, 200, 2500)
        
        return rr_intervals
    
    def generate_bradycardia_hrv(self, base_hr=45):
        """
        Generate bradycardia HRV pattern
        - Slow heart rate (< 60 bpm)
        - Higher variability due to increased parasympathetic activity
        """
        base_rr = 60000 / base_hr
        
        # Enhanced respiratory variation
        respiratory_freq = 0.2
        time_points = np.linspace(0, self.sequence_length/self.sampling_rate, self.sequence_length)
        respiratory_variation = 80 * np.sin(2 * np.pi * respiratory_freq * time_points)
        
        # Additional slow wave variation
        slow_variation = 40 * np.sin(2 * np.pi * 0.1 * time_points)
        
        random_variation = np.random.normal(0, 25, self.sequence_length)
        
        rr_intervals = base_rr + respiratory_variation + slow_variation + random_variation
        rr_intervals = np.clip(rr_intervals, 600, 2500)
        
        return rr_intervals
    
    def generate_tachycardia_hrv(self, base_hr=120):
        """
        Generate tachycardia HRV pattern
        - Fast heart rate (> 100 bpm)
        - Reduced variability due to increased sympathetic activity
        """
        base_rr = 60000 / base_hr
        
        # Reduced respiratory variation
        respiratory_freq = 0.3
        time_points = np.linspace(0, self.sequence_length/self.sampling_rate, self.sequence_length)
        respiratory_variation = 10 * np.sin(2 * np.pi * respiratory_freq * time_points)
        
        # Small random variation
        random_variation = np.random.normal(0, 8, self.sequence_length)
        
        rr_intervals = base_rr + respiratory_variation + random_variation
        rr_intervals = np.clip(rr_intervals, 300, 1000)
        
        return rr_intervals
    
    def add_apple_watch_noise(self, rr_intervals):
        """
        Add realistic Apple Watch measurement noise and artifacts
        """
        # Add measurement noise (±2-5ms typical for optical sensors)
        noise = np.random.normal(0, 3, len(rr_intervals))
        
        # Add occasional motion artifacts (sudden spikes)
        artifact_probability = 0.02  # 2% chance of artifact per sample
        artifacts = np.random.binomial(1, artifact_probability, len(rr_intervals))
        artifact_magnitude = np.random.normal(0, 20, len(rr_intervals)) * artifacts
        
        return rr_intervals + noise + artifact_magnitude

# Initialize the generator
hrv_generator = HRVDataGenerator(sequence_length=200)

# Generate synthetic dataset
num_samples_per_class = 2500
total_samples = num_samples_per_class * 4

print(f"Generating {total_samples} HRV sequences...")

# Create arrays to store data
hrv_data = []
labels = []
label_names = ['Normal', 'Atrial Fibrillation', 'Bradycardia', 'Tachycardia']

# Generate each class
for class_idx, (class_name, generator_func) in enumerate([
    ('Normal', hrv_generator.generate_normal_hrv),
    ('Atrial Fibrillation', hrv_generator.generate_afib_hrv),
    ('Bradycardia', hrv_generator.generate_bradycardia_hrv),
    ('Tachycardia', hrv_generator.generate_tachycardia_hrv)
]):
    print(f"Generating {class_name} patterns...")
    
    for i in range(num_samples_per_class):
        # Generate base pattern
        if class_name == 'Normal':
            base_hr = np.random.normal(75, 10)  # Vary base heart rate
            base_hr = np.clip(base_hr, 60, 100)
            rr_sequence = generator_func(base_hr)
        elif class_name == 'Atrial Fibrillation':
            base_hr = np.random.normal(90, 15)
            base_hr = np.clip(base_hr, 70, 150)
            rr_sequence = generator_func(base_hr)
        elif class_name == 'Bradycardia':
            base_hr = np.random.normal(50, 8)
            base_hr = np.clip(base_hr, 35, 60)
            rr_sequence = generator_func(base_hr)
        else:  # Tachycardia
            base_hr = np.random.normal(130, 20)
            base_hr = np.clip(base_hr, 100, 180)
            rr_sequence = generator_func(base_hr)
        
        # Add Apple Watch-like noise
        rr_sequence = hrv_generator.add_apple_watch_noise(rr_sequence)
        
        hrv_data.append(rr_sequence)
        labels.append(class_idx)

# Convert to numpy arrays
hrv_data = np.array(hrv_data)
labels = np.array(labels)

print(f"Generated dataset shape: {hrv_data.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Class distribution: {np.bincount(labels)}")

## Step 2: Data Preprocessing and Feature Engineering for HRV

In [ ]:
def compute_hrv_features(rr_intervals):
    """
    Compute time-domain and frequency-domain HRV features
    """
    # Time-domain features
    mean_rr = np.mean(rr_intervals)
    std_rr = np.std(rr_intervals)
    
    # RMSSD (Root Mean Square of Successive Differences)
    successive_diffs = np.diff(rr_intervals)
    rmssd = np.sqrt(np.mean(successive_diffs**2))
    
    # pNN50 (percentage of successive RR intervals that differ by > 50ms)
    nn50_count = np.sum(np.abs(successive_diffs) > 50)
    pnn50 = (nn50_count / len(successive_diffs)) * 100
    
    return [mean_rr, std_rr, rmssd, pnn50]

def preprocess_hrv_data(hrv_sequences):
    """
    Preprocess HRV data for CNN input
    """
    processed_sequences = []
    
    for sequence in hrv_sequences:
        # Convert RR intervals to heart rate (BPM)
        hr_sequence = 60000 / sequence  # Convert ms to BPM
        
        # Compute RR interval differences (captures variability)
        rr_diffs = np.diff(sequence)
        # Pad to maintain sequence length
        rr_diffs = np.pad(rr_diffs, (0, 1), mode='edge')
        
        # Stack features: [RR intervals, HR, RR differences]
        features = np.stack([sequence, hr_sequence, rr_diffs], axis=-1)
        processed_sequences.append(features)
    
    return np.array(processed_sequences)

# Preprocess the data
print("Preprocessing HRV data...")
X_processed = preprocess_hrv_data(hrv_data)

# Normalize the features
scaler = StandardScaler()
original_shape = X_processed.shape
X_normalized = scaler.fit_transform(X_processed.reshape(-1, X_processed.shape[-1]))
X_normalized = X_normalized.reshape(original_shape)

# Convert labels to categorical
y_categorical = tf.keras.utils.to_categorical(labels, num_classes=4)

print(f"Processed data shape: {X_normalized.shape}")
print(f"Categorical labels shape: {y_categorical.shape}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y_categorical, test_size=0.2, random_state=42, stratify=labels
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape: {X_test.shape}")

## Step 3: Visualize HRV Patterns

In [ ]:
# Visualize sample patterns from each class
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('HRV Patterns for Different Heart Conditions', fontsize=16)

for class_idx in range(4):
    # Find first sample of this class
    sample_idx = np.where(labels == class_idx)[0][0]
    sample_data = hrv_data[sample_idx]
    
    # Plot RR intervals
    axes[0, class_idx].plot(sample_data, 'b-', linewidth=1)
    axes[0, class_idx].set_title(f'{label_names[class_idx]}\nRR Intervals')
    axes[0, class_idx].set_ylabel('RR Interval (ms)')
    axes[0, class_idx].grid(True, alpha=0.3)
    
    # Plot heart rate
    hr_data = 60000 / sample_data
    axes[1, class_idx].plot(hr_data, 'r-', linewidth=1)
    axes[1, class_idx].set_title('Heart Rate')
    axes[1, class_idx].set_ylabel('Heart Rate (BPM)')
    axes[1, class_idx].set_xlabel('Time (samples)')
    axes[1, class_idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show HRV statistics for each class
print("\nHRV Statistics by Class:")
print("=" * 50)

for class_idx in range(4):
    class_samples = hrv_data[labels == class_idx]
    
    # Compute statistics
    mean_rr = np.mean([np.mean(sample) for sample in class_samples])
    mean_hr = 60000 / mean_rr
    mean_rmssd = np.mean([np.sqrt(np.mean(np.diff(sample)**2)) for sample in class_samples])
    
    print(f"\n{label_names[class_idx]}:")
    print(f"  Mean RR Interval: {mean_rr:.1f} ms")
    print(f"  Mean Heart Rate: {mean_hr:.1f} BPM")
    print(f"  Mean RMSSD: {mean_rmssd:.1f} ms")

## Step 4: Build Advanced CNN Architecture for HRV Pattern Recognition

In [ ]:
def build_hrv_cnn_model(input_shape, num_classes=4):
    """
    Build advanced CNN architecture optimized for HRV pattern recognition
    
    Architecture features:
    - Multi-scale convolutions to capture different temporal patterns
    - Residual connections for better gradient flow
    - Attention mechanism to focus on important time segments
    - Dropout and batch normalization for regularization
    """
    
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # Multi-scale feature extraction
    # Short-term patterns (kernel size 3-5)
    conv1_short = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(inputs)
    conv1_short = tf.keras.layers.BatchNormalization()(conv1_short)
    
    conv2_short = tf.keras.layers.Conv1D(32, 5, activation='relu', padding='same')(inputs)
    conv2_short = tf.keras.layers.BatchNormalization()(conv2_short)
    
    # Medium-term patterns (kernel size 7-11)
    conv1_med = tf.keras.layers.Conv1D(32, 7, activation='relu', padding='same')(inputs)
    conv1_med = tf.keras.layers.BatchNormalization()(conv1_med)
    
    conv2_med = tf.keras.layers.Conv1D(32, 11, activation='relu', padding='same')(inputs)
    conv2_med = tf.keras.layers.BatchNormalization()(conv2_med)
    
    # Concatenate multi-scale features
    multi_scale = tf.keras.layers.Concatenate()([conv1_short, conv2_short, conv1_med, conv2_med])
    
    # First residual block
    res1 = tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same')(multi_scale)
    res1 = tf.keras.layers.BatchNormalization()(res1)
    res1 = tf.keras.layers.Conv1D(64, 3, padding='same')(res1)
    res1 = tf.keras.layers.BatchNormalization()(res1)
    
    # Adjust dimensions for residual connection
    multi_scale_proj = tf.keras.layers.Conv1D(64, 1, padding='same')(multi_scale)
    res1_output = tf.keras.layers.Add()([res1, multi_scale_proj])
    res1_output = tf.keras.layers.Activation('relu')(res1_output)
    res1_output = tf.keras.layers.MaxPooling1D(2)(res1_output)
    res1_output = tf.keras.layers.Dropout(0.2)(res1_output)
    
    # Second residual block
    res2 = tf.keras.layers.Conv1D(128, 3, activation='relu', padding='same')(res1_output)
    res2 = tf.keras.layers.BatchNormalization()(res2)
    res2 = tf.keras.layers.Conv1D(128, 3, padding='same')(res2)
    res2 = tf.keras.layers.BatchNormalization()(res2)
    
    res1_proj = tf.keras.layers.Conv1D(128, 1, padding='same')(res1_output)
    res2_output = tf.keras.layers.Add()([res2, res1_proj])
    res2_output = tf.keras.layers.Activation('relu')(res2_output)
    res2_output = tf.keras.layers.MaxPooling1D(2)(res2_output)
    res2_output = tf.keras.layers.Dropout(0.3)(res2_output)
    
    # Attention mechanism
    attention_weights = tf.keras.layers.Conv1D(1, 1, activation='softmax', padding='same')(res2_output)
    attended_features = tf.keras.layers.Multiply()([res2_output, attention_weights])
    
    # Global average pooling and max pooling
    gap = tf.keras.layers.GlobalAveragePooling1D()(attended_features)
    gmp = tf.keras.layers.GlobalMaxPooling1D()(attended_features)
    
    # Combine pooled features
    combined = tf.keras.layers.Concatenate()([gap, gmp])
    
    # Dense layers with dropout
    dense1 = tf.keras.layers.Dense(256, activation='relu')(combined)
    dense1 = tf.keras.layers.BatchNormalization()(dense1)
    dense1 = tf.keras.layers.Dropout(0.4)(dense1)
    
    dense2 = tf.keras.layers.Dense(128, activation='relu')(dense1)
    dense2 = tf.keras.layers.BatchNormalization()(dense2)
    dense2 = tf.keras.layers.Dropout(0.3)(dense2)
    
    # Output layer
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(dense2)
    
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    
    return model

# Build the model
input_shape = X_train.shape[1:]
model = build_hrv_cnn_model(input_shape, num_classes=4)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

# Display model architecture
model.summary()

# Plot model architecture
tf.keras.utils.plot_model(
    model, 
    to_file='hrv_cnn_architecture.png', 
    show_shapes=True, 
    show_layer_names=True,
    rankdir='TB'
)

print(f"\nModel built successfully!")
print(f"Total parameters: {model.count_params():,}")

## Step 5: Train the HRV CNN Model

In [ ]:
# Define callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=8,
        min_lr=1e-7,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_hrv_cnn_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("Starting model training...")
print("This may take several minutes depending on your hardware.")

# Train the model
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("\nTraining completed!")

## Step 6: Evaluate Model Performance

In [ ]:
# Load the best model
best_model = tf.keras.models.load_model('best_hrv_cnn_model.h5')

# Evaluate on test set
print("Evaluating model on test set...")
test_loss, test_accuracy, test_precision, test_recall = best_model.evaluate(X_test, y_test, verbose=0)

print(f"\nTest Results:")
print(f"Loss: {test_loss:.4f}")
print(f"Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-Score: {2 * (test_precision * test_recall) / (test_precision + test_recall):.4f}")

# Get predictions
y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Detailed classification report
print("\nDetailed Classification Report:")
print("=" * 50)
print(classification_report(y_test_classes, y_pred_classes, target_names=label_names))

# Confusion Matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_names, yticklabels=label_names)
plt.title('Confusion Matrix - HRV Pattern Classification')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Training History', fontsize=16)

# Accuracy
axes[0, 0].plot(history.history['accuracy'], label='Training Accuracy')
axes[0, 0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[0, 0].set_title('Model Accuracy')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Loss
axes[0, 1].plot(history.history['loss'], label='Training Loss')
axes[0, 1].plot(history.history['val_loss'], label='Validation Loss')
axes[0, 1].set_title('Model Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Precision
axes[1, 0].plot(history.history['precision'], label='Training Precision')
axes[1, 0].plot(history.history['val_precision'], label='Validation Precision')
axes[1, 0].set_title('Model Precision')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Recall
axes[1, 1].plot(history.history['recall'], label='Training Recall')
axes[1, 1].plot(history.history['val_recall'], label='Validation Recall')
axes[1, 1].set_title('Model Recall')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 7: Apple Watch Integration Functions

In [ ]:
class AppleWatchHRVAnalyzer:
    """
    Class for analyzing Apple Watch HRV data in real-time
    """
    
    def __init__(self, model_path='best_hrv_cnn_model.h5'):
        self.model = tf.keras.models.load_model(model_path)
        self.scaler = scaler  # Use the fitted scaler from training
        self.label_names = ['Normal', 'Atrial Fibrillation', 'Bradycardia', 'Tachycardia']
        self.sequence_length = 200
        
    def preprocess_apple_watch_data(self, heart_rate_data, timestamps):
        """
        Convert Apple Watch heart rate data to RR intervals
        
        Args:
            heart_rate_data: List of heart rate values in BPM
            timestamps: List of timestamps for each measurement
        
        Returns:
            RR intervals in milliseconds
        """
        # Convert HR to RR intervals
        rr_intervals = 60000 / np.array(heart_rate_data)  # Convert to ms
        
        # Apply smoothing to reduce noise
        if len(rr_intervals) > 5:
            rr_intervals = signal.savgol_filter(rr_intervals, 5, 3)
        
        return rr_intervals
    
    def create_sliding_windows(self, rr_intervals, window_size=200, overlap=0.5):
        """
        Create sliding windows from continuous RR interval data
        """
        step_size = int(window_size * (1 - overlap))
        windows = []
        
        for i in range(0, len(rr_intervals) - window_size + 1, step_size):
            window = rr_intervals[i:i + window_size]
            windows.append(window)
        
        return np.array(windows)
    
    def analyze_hrv_pattern(self, rr_intervals):
        """
        Analyze HRV pattern and return prediction with confidence
        """
        if len(rr_intervals) < self.sequence_length:
            # Pad with mean if sequence is too short
            mean_rr = np.mean(rr_intervals)
            padded = np.pad(rr_intervals, (0, self.sequence_length - len(rr_intervals)), 
                          mode='constant', constant_values=mean_rr)
            rr_intervals = padded
        elif len(rr_intervals) > self.sequence_length:
            # Take the most recent data
            rr_intervals = rr_intervals[-self.sequence_length:]
        
        # Preprocess similar to training data
        hr_sequence = 60000 / rr_intervals
        rr_diffs = np.diff(rr_intervals)
        rr_diffs = np.pad(rr_diffs, (0, 1), mode='edge')
        
        # Stack features
        features = np.stack([rr_intervals, hr_sequence, rr_diffs], axis=-1)
        features = features.reshape(1, *features.shape)
        
        # Normalize
        original_shape = features.shape
        features_normalized = self.scaler.transform(features.reshape(-1, features.shape[-1]))
        features_normalized = features_normalized.reshape(original_shape)
        
        # Predict
        prediction = self.model.predict(features_normalized, verbose=0)
        
        # Get predicted class and confidence
        predicted_class = np.argmax(prediction[0])
        confidence = np.max(prediction[0])
        
        return {
            'predicted_condition': self.label_names[predicted_class],
            'confidence': confidence,
            'probabilities': dict(zip(self.label_names, prediction[0])),
            'risk_level': self.get_risk_level(predicted_class, confidence)
        }
    
    def get_risk_level(self, predicted_class, confidence):
        """
        Determine risk level based on prediction and confidence
        """
        if predicted_class == 0:  # Normal
            return 'Low' if confidence > 0.8 else 'Moderate'
        elif predicted_class == 1:  # Atrial Fibrillation
            return 'High' if confidence > 0.7 else 'Moderate'
        elif predicted_class == 2:  # Bradycardia
            return 'Moderate' if confidence > 0.6 else 'Low'
        else:  # Tachycardia
            return 'Moderate' if confidence > 0.6 else 'Low'
    
    def generate_health_report(self, rr_intervals, analysis_result):
        """
        Generate a comprehensive health report
        """
        # Calculate HRV metrics
        mean_rr = np.mean(rr_intervals)
        mean_hr = 60000 / mean_rr
        rmssd = np.sqrt(np.mean(np.diff(rr_intervals)**2))
        pnn50 = (np.sum(np.abs(np.diff(rr_intervals)) > 50) / len(np.diff(rr_intervals))) * 100
        
        report = {
            'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
            'hrv_metrics': {
                'mean_heart_rate_bpm': round(mean_hr, 1),
                'mean_rr_interval_ms': round(mean_rr, 1),
                'rmssd_ms': round(rmssd, 1),
                'pnn50_percent': round(pnn50, 1)
            },
            'prediction': analysis_result,
            'recommendations': self.get_recommendations(analysis_result)
        }
        
        return report
    
    def get_recommendations(self, analysis_result):
        """
        Provide health recommendations based on analysis
        """
        condition = analysis_result['predicted_condition']
        risk_level = analysis_result['risk_level']
        
        recommendations = []
        
        if condition == 'Normal':
            recommendations = [
                "Your heart rhythm appears normal. Continue maintaining a healthy lifestyle.",
                "Regular exercise and stress management support good heart health.",
                "Continue monitoring your heart rate variability."
            ]
        elif condition == 'Atrial Fibrillation':
            recommendations = [
                "⚠️ Irregular heart rhythm detected. Consider consulting a healthcare provider.",
                "Avoid excessive caffeine and alcohol consumption.",
                "Monitor symptoms like palpitations, shortness of breath, or chest discomfort.",
                "Keep a record of when irregular rhythms occur."
            ]
        elif condition == 'Bradycardia':
            recommendations = [
                "Slow heart rate detected. Monitor for symptoms like fatigue or dizziness.",
                "If you're an athlete, this may be normal. Otherwise, consider medical evaluation.",
                "Stay hydrated and avoid sudden position changes."
            ]
        elif condition == 'Tachycardia':
            recommendations = [
                "Elevated heart rate detected. Consider factors like stress, caffeine, or physical activity.",
                "Practice relaxation techniques and ensure adequate rest.",
                "If persistent without clear cause, consult a healthcare provider."
            ]
        
        if risk_level == 'High':
            recommendations.insert(0, "🚨 HIGH RISK: Seek immediate medical attention if experiencing symptoms.")
        
        return recommendations

# Initialize the analyzer
hrv_analyzer = AppleWatchHRVAnalyzer()

print("Apple Watch HRV Analyzer initialized successfully!")
print("Ready to analyze heart rate variability patterns.")

## Step 8: Demonstration with Simulated Apple Watch Data

In [ ]:
# Simulate Apple Watch heart rate data for different scenarios
def simulate_apple_watch_data(condition='normal', duration_minutes=5):
    """
    Simulate Apple Watch heart rate data
    """
    # Apple Watch typically samples every 15-60 seconds during passive monitoring
    # During workout mode, it can sample every 1-5 seconds
    sampling_interval = 15  # seconds
    num_samples = duration_minutes * 60 // sampling_interval
    
    timestamps = pd.date_range(
        start=pd.Timestamp.now() - pd.Timedelta(minutes=duration_minutes),
        periods=num_samples,
        freq=f'{sampling_interval}S'
    )
    
    if condition == 'normal':
        base_hr = 72
        hr_data = base_hr + np.random.normal(0, 8, num_samples)
        # Add some natural variation
        hr_data += 5 * np.sin(np.linspace(0, 4*np.pi, num_samples))  # Respiratory variation
        
    elif condition == 'afib':
        base_hr = 85
        hr_data = base_hr + np.random.normal(0, 20, num_samples)
        # Add irregular spikes
        for i in range(0, num_samples, 5):
            if np.random.random() < 0.3:  # 30% chance of irregular beat
                hr_data[i:i+2] += np.random.normal(15, 10, min(2, num_samples-i))
                
    elif condition == 'bradycardia':
        base_hr = 48
        hr_data = base_hr + np.random.normal(0, 6, num_samples)
        # Higher variability
        hr_data += 8 * np.sin(np.linspace(0, 2*np.pi, num_samples))
        
    elif condition == 'tachycardia':
        base_hr = 130
        hr_data = base_hr + np.random.normal(0, 5, num_samples)
        # Less variability
        hr_data += 2 * np.sin(np.linspace(0, np.pi, num_samples))
    
    # Ensure physiologically reasonable values
    hr_data = np.clip(hr_data, 30, 220)
    
    return hr_data, timestamps

# Demonstrate analysis for different conditions
conditions = ['normal', 'afib', 'bradycardia', 'tachycardia']
condition_names = ['Normal', 'Atrial Fibrillation', 'Bradycardia', 'Tachycardia']

print("🍎 Apple Watch HRV Analysis Demonstration")
print("=" * 50)

for i, condition in enumerate(conditions):
    print(f"\n📱 Simulating {condition_names[i]} condition...")
    
    # Generate simulated data
    hr_data, timestamps = simulate_apple_watch_data(condition, duration_minutes=10)
    
    # Convert to RR intervals
    rr_intervals = hrv_analyzer.preprocess_apple_watch_data(hr_data, timestamps)
    
    # Analyze the pattern
    analysis_result = hrv_analyzer.analyze_hrv_pattern(rr_intervals)
    
    # Generate comprehensive report
    health_report = hrv_analyzer.generate_health_report(rr_intervals, analysis_result)
    
    # Display results
    print(f"\n📊 Analysis Results for {condition_names[i]}:")
    print(f"Predicted Condition: {analysis_result['predicted_condition']}")
    print(f"Confidence: {analysis_result['confidence']:.1%}")
    print(f"Risk Level: {analysis_result['risk_level']}")
    
    print(f"\n💓 HRV Metrics:")
    metrics = health_report['hrv_metrics']
    print(f"Mean Heart Rate: {metrics['mean_heart_rate_bpm']} BPM")
    print(f"RMSSD: {metrics['rmssd_ms']} ms")
    print(f"pNN50: {metrics['pnn50_percent']}%")
    
    print(f"\n💡 Recommendations:")
    for rec in health_report['recommendations']:
        print(f"  • {rec}")
    
    print("-" * 50)

## Step 9: Real-time Monitoring Simulation

In [ ]:
# Simulate real-time monitoring
def simulate_realtime_monitoring(duration_minutes=30, condition_changes=None):
    """
    Simulate real-time HRV monitoring with condition changes
    
    Args:
        duration_minutes: Total monitoring duration
        condition_changes: List of (time_minute, new_condition) tuples
    """
    if condition_changes is None:
        condition_changes = [
            (0, 'normal'),
            (10, 'tachycardia'),  # Stress/exercise starts
            (20, 'afib'),         # Irregular rhythm develops
            (25, 'normal')        # Returns to normal
        ]
    
    print("🔄 Real-time HRV Monitoring Simulation")
    print(f"Duration: {duration_minutes} minutes")
    print("=" * 60)
    
    all_hr_data = []
    all_timestamps = []
    condition_log = []
    
    # Generate data for each condition period
    for i, (start_time, condition) in enumerate(condition_changes):
        # Determine duration for this condition
        if i < len(condition_changes) - 1:
            end_time = condition_changes[i + 1][0]
        else:
            end_time = duration_minutes
        
        period_duration = end_time - start_time
        
        if period_duration > 0:
            hr_data, timestamps = simulate_apple_watch_data(condition, period_duration)
            
            # Adjust timestamps to be relative to monitoring start
            adjusted_timestamps = timestamps + pd.Timedelta(minutes=start_time)
            
            all_hr_data.extend(hr_data)
            all_timestamps.extend(adjusted_timestamps)
            condition_log.append((start_time, condition, len(hr_data)))
    
    # Analyze the complete data in sliding windows
    all_hr_data = np.array(all_hr_data)
    rr_intervals = hrv_analyzer.preprocess_apple_watch_data(all_hr_data, all_timestamps)
    
    # Create sliding windows for analysis
    windows = hrv_analyzer.create_sliding_windows(rr_intervals, window_size=200, overlap=0.7)
    
    print(f"\n📈 Analyzing {len(windows)} time windows...")
    
    # Analyze each window
    results = []
    for i, window in enumerate(windows):
        analysis = hrv_analyzer.analyze_hrv_pattern(window)
        
        window_time = i * (200 * 0.3) / 4  # Approximate time in minutes
        results.append({
            'time_minutes': round(window_time, 1),
            'condition': analysis['predicted_condition'],
            'confidence': analysis['confidence'],
            'risk_level': analysis['risk_level']
        })
    
    # Display timeline
    print("\n⏰ Timeline Analysis:")
    print(f"{'Time (min)':<10} {'Detected Condition':<18} {'Confidence':<12} {'Risk Level':<10}")
    print("-" * 60)
    
    for result in results[::3]:  # Show every 3rd result to avoid clutter
        print(f"{result['time_minutes']:<10} {result['condition']:<18} {result['confidence']:<12.1%} {result['risk_level']:<10}")
    
    # Summary statistics
    condition_counts = {}
    for result in results:
        condition = result['condition']
        condition_counts[condition] = condition_counts.get(condition, 0) + 1
    
    print(f"\n📊 Summary:")
    print(f"Total analysis windows: {len(results)}")
    print(f"Condition distribution:")
    for condition, count in condition_counts.items():
        percentage = (count / len(results)) * 100
        print(f"  {condition}: {count} windows ({percentage:.1f}%)")
    
    # High-risk periods
    high_risk_periods = [r for r in results if r['risk_level'] == 'High']
    if high_risk_periods:
        print(f"\n⚠️ High-risk periods detected: {len(high_risk_periods)}")
        for period in high_risk_periods[:5]:  # Show first 5
            print(f"  Time {period['time_minutes']} min: {period['condition']} ({period['confidence']:.1%})")
    
    return results

# Run real-time monitoring simulation
monitoring_results = simulate_realtime_monitoring(duration_minutes=30)

## Step 10: Model Saving and Export

In [ ]:
# Save the complete model and preprocessing components
import pickle
import json

print("💾 Saving model and components...")

# Save the trained model
model_save_path = 'apple_watch_hrv_cnn_model.h5'
best_model.save(model_save_path)
print(f"✅ Model saved to: {model_save_path}")

# Save the scaler
scaler_save_path = 'hrv_scaler.pickle'
with open(scaler_save_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"✅ Scaler saved to: {scaler_save_path}")

# Save model metadata
model_metadata = {
    'model_version': '1.0',
    'created_date': pd.Timestamp.now().isoformat(),
    'model_type': 'CNN for HRV Pattern Recognition',
    'input_shape': input_shape,
    'num_classes': 4,
    'class_names': label_names,
    'sequence_length': 200,
    'features': ['RR intervals (ms)', 'Heart Rate (BPM)', 'RR differences (ms)'],
    'test_accuracy': float(test_accuracy),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'total_parameters': int(best_model.count_params()),
    'training_samples': len(X_train),
    'validation_samples': len(X_val),
    'test_samples': len(X_test)
}

metadata_save_path = 'model_metadata.json'
with open(metadata_save_path, 'w') as f:
    json.dump(model_metadata, f, indent=2)
print(f"✅ Metadata saved to: {metadata_save_path}")

# Create a deployment package info
deployment_info = """
🚀 Apple Watch HRV CNN Model - Deployment Package

Files included:
1. apple_watch_hrv_cnn_model.h5 - Trained CNN model
2. hrv_scaler.pickle - Feature scaler for preprocessing
3. model_metadata.json - Model information and performance metrics
4. HRV_CNN_Analysis.ipynb - Complete training notebook

Usage:
1. Load the model: tf.keras.models.load_model('apple_watch_hrv_cnn_model.h5')
2. Load the scaler: pickle.load(open('hrv_scaler.pickle', 'rb'))
3. Use AppleWatchHRVAnalyzer class for inference

Model Performance:
- Test Accuracy: {:.2%}
- Test Precision: {:.2%}
- Test Recall: {:.2%}
- Total Parameters: {:,}

Supported Conditions:
- Normal heart rhythm
- Atrial Fibrillation
- Bradycardia
- Tachycardia

Input Requirements:
- Heart rate data from Apple Watch (continuous monitoring)
- Minimum 200 RR intervals for reliable analysis
- Sampling rate: ~4 Hz (typical for Apple Watch)

Integration Notes:
- Compatible with HealthKit for Apple Watch data
- Real-time analysis with sliding window approach
- Risk level assessment and health recommendations
- Suitable for continuous monitoring applications
""".format(
    test_accuracy, test_precision, test_recall, best_model.count_params()
)

with open('DEPLOYMENT_INFO.txt', 'w') as f:
    f.write(deployment_info)

print(f"✅ Deployment info saved to: DEPLOYMENT_INFO.txt")

print("\n🎉 Model training and saving completed successfully!")
print("\n📱 Your Apple Watch HRV analysis model is ready for deployment.")
print(f"\n📊 Final Model Performance:")
print(f"   Accuracy: {test_accuracy:.2%}")
print(f"   Precision: {test_precision:.2%}")
print(f"   Recall: {test_recall:.2%}")
print(f"   Parameters: {best_model.count_params():,}")